# "বাংলাদেশের কোন জেলা লোকডাউন মেনে চলছে"
- toc: false
- branch: master
- badges: true
- comments: true
- categories: [mobility, facebook, bangladesh]
- image: images/district-stay-put.png
- hide: false
- search_exclude: true
- metadata_key1: mobility
- metadata_key2: covid-19

In [13]:
#hide_input
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
import pandas as pd
import altair as alt

In [24]:
#hide_input
country_name = "Bangladesh"
country_code = "BGD"
df = pd.read_csv('movement-range-2020-07-10.txt', \
                 sep='\t', parse_dates=['ds'], low_memory=False)
bdf = df[df['country'] == country_code]

change_col = 'all_day_bing_tiles_visited_relative_change'
stay_put_col = 'all_day_ratio_single_tile_users'

bn_change_col_name = 'ফেব্রুয়ারী ২০২০ থেকে কতখানি পরিবর্তন'
bn_stay_put_col_name = 'কতোভাগ মানুষ বাড়িতে থাকছে'

#bdf.tail()

কোভিড-১৯ প্যান্ডেমিক এর কারণে মানুষের চলাফেরা তে কি পরিবর্তন এসেছে তার একটি [ডেটাসেট](https://data.humdata.org/dataset/movement-range-maps) ফেইসবুক প্রকাশ করেছে। ফেইসবুক ব্যবহারকারীদের ওয়াইফাই এর ব্যবহার পর্যবেক্ষণ করে ডেটাসেট বানানো হয়েছে। এখানে দেখা হয়েছে লোকেরা বাড়ির তে  অবস্থান করছে, নাকি শহরে ঘুরে বেড়াচ্ছে। এটা প্রতিরোধমূলক ব্যবস্থাগুলি সঠিক পথে পরিচালিত হচ্ছে কিনা সে ব্যাপারে অবহিত করতে পারে।
এই পোস্টে বাংলাদেশের ৬৪ জেলার ডেটা তুলে ধরা হল। 

[ডেটা](https://data.humdata.org/dataset/movement-range-maps)

In [28]:
#hide_input
bdf_latest_date = bdf[bdf['ds'] == bdf['ds'].max()]
bdf_latest_date = bdf_latest_date.sort_values(by=[stay_put_col], ascending=False)

color_var=alt.Color('Type:N', scale=alt.Scale(scheme='tableau20'), legend=None)
barchart = alt.Chart(bdf_latest_date).mark_bar().encode(
    y=alt.Y('polygon_name', axis=alt.Axis(title='জেলা'), sort='-x'),
    x=alt.X(stay_put_col, axis=alt.Axis(title=bn_stay_put_col_name, format='%')),
    color=color_var,
    tooltip=['polygon_name', stay_put_col],
    
).properties(
    title = bn_stay_put_col_name
)

barchart.properties(width=600, height=600)
#barchart.save('images/district-stay-put.png')

alt.Chart(...)

In [26]:
#hide_input
line = alt.Chart(bdf_latest_date).mark_line(color='red').encode(
    x=alt.X('polygon_name', axis=alt.Axis(title='জেলা'), sort='y'),
    y=alt.Y(change_col, axis=alt.Axis(title=bn_change_col_name, format='%')),
    tooltip=['polygon_name', change_col],
).properties(
    title="কোভিড-১৯ প্যান্ডেমিক শুরুর আগে ফেব্রুয়ারী ২০২০ এর সাথে তুলোনা"
)
line.properties(width=700, height=300)

alt.Chart(...)

In [31]:
#hide_input

# Prepare data
cities = ['Dhaka', 'Narayanganj', 'Chittagong', 'Sylhet', 'Rajshahi', 'Khulna', 'Barisal', 'Gazipur']
cities_data = []
for acity in cities:
    city_data = bdf.loc[bdf['polygon_name'] == acity]
    city_data = city_data.copy()
    city_data['City'] = acity
    cities_data.append(city_data)

cities_data = pd.concat(cities_data)
cities_data.tail()

color_var=alt.Color('City:N', scale=alt.Scale(scheme='tableau20'), legend=None)
chart = alt.Chart(cities_data).mark_line().encode(
    x=alt.X('monthdate(ds):O', axis=alt.Axis(title='Date')),
    y=alt.Y(stay_put_col, axis=alt.Axis(title=bn_stay_put_col_name, format='%')),
    color=color_var,
    tooltip=['monthdate(ds):O', 'City:N', stay_put_col],
    
).properties(
    title=f"শহরগুলোতে {bn_stay_put_col_name}"
)

legend = alt.Chart(cities_data).mark_point().encode(
    y=alt.Y('City:N', axis=alt.Axis(orient='right')),
    color=color_var
)
chart.properties(width=600, height=300)|legend


alt.HConcatChart(...)

In [18]:
#hide_input
# getting data from https://data.humdata.org/dataset/movement-range-maps
#!wget https://data.humdata.org/dataset/c3429f0e-651b-4788-bb2f-4adbf222c90e/resource/31ca909c-10d9-458a-8720-88b54b3e3627/download/movement-range-data-2020-06-22.zip
#!ls 
#!yes "yes" | unzip movement-range-data-2020-06-22.zip
# caution 
#!rm -rf movement-range-data-2020-06-10 && rm -rf movement-range-data-2020-06-10.zip
#!head -n 30 movement-range-2020-06-22.txt